**CYBER SMILE , CHILD AND GROOMED COMMENT STORIES , CHILD ABUSE HASHTAGS**

In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

In [2]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn import metrics
from sklearn.metrics import classification_report 
import textblob
from textblob import TextBlob

[nltk_data] Downloading package stopwords to C:\Users\Akhil
[nltk_data]     Sanker\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Akhil
[nltk_data]     Sanker\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [3]:
chat_slang = pd.read_excel("Chat_Slang_Dictionary.xlsx")

In [4]:
chat_slang
slang_dict = dict(zip(chat_slang['Chat Term'],chat_slang['Translated Term'])) #create a dictionary for swapping later as part of 
#preprocessing
slang = list(slang_dict.keys())

In [5]:
#adding additonal to chat_slang
slang_dict["cp"] = "child porn"

In [6]:
def replace_slang(string):
    global slang , slang_dict
    for s in slang:
        if s in string.lower().split():
            string = string.replace(s,slang_dict[s])
        else:
            pass
    return string 
        

In [7]:
si = ['\n','.',',','"',';',':','\t']
def clean_base(string):
    for s in si:
        
        if s in string.lower():

            string = string.replace(s," ")
        else:
            pass
    return string

In [8]:
cyber_smile = pd.read_csv("CyberSmile data scrap.csv")
chat_groom_comment = pd.read_csv("child AND groomed_comments_stories.csv")
chat_hash_tags = pd.read_csv("Child_abuse_hashtags_cleaned.csv")
good_temp = pd.read_csv("good_data.csv")

In [9]:
good_temp.drop(['id','qid1',
               'qid2','question2','is_duplicate'],axis=1,inplace=True)

In [10]:
good_temp = good_temp.iloc[:250].reset_index(drop=True)
good_temp['target'] = [1 for x in range(good_temp.shape[0])]
good_temp = good_temp.rename(columns={"question1":"Content"})

***CyberSmile - Analysis***

In [11]:
cs=cyber_smile

In [12]:
try:
    cs.drop(['Thread_url'],inplace=True,axis=1)
except:
    pass

In [13]:
cs

,Thread_name,Thread_Content
0,"I need help, I don't know what to do. It won't...",This guy that harassed me back in 2007 is hara...
1,Instagram anonymous messages,"Hi there,\nI’m in my 30s and getting cyber bul..."
2,Doxing,"Hi, I was recently doxed by a few people who k..."
3,I feel hopeless right now,I've had a group of ppl harassing me for over ...
4,Stop Cyberbullying Day 2020,Just wondering how this year went with everyon...
...,...,...
244,"You are the badest person ever die"" Yep i was ...",There was more stuff but it was swearing can s...
245,How do I protect myself online?,I have been cyber bullied before and I got ove...
246,Sexting - advice for my daughters,I have two daughters 8 and 14 and i am very wo...
247,Instagram troubles,"Someone is abusing my friend on Instagram, wha..."


In [14]:
name_list = []
for i in range(cs.shape[0]):
    new_str = clean_base(cs['Thread_name'][i])
    new_str = replace_slang(new_str)
    name_list.append(new_str)
Content = []
for i in range(cs.shape[0]):
    new_str = clean_base(cs['Thread_Content'][i])
    new_str = replace_slang(new_str)
    Content.append(new_str)

In [15]:
try:
    cs['Name'] = name_list
    cs['Content'] = Content
    cs.drop(['Thread_name','Thread_Content'],axis=1,inplace=True)
    cs['target'] = [0 for x in range(cs.shape[0])]
except:
    pass

In [16]:
cs = pd.concat([cs,good_temp])
#cs.drop(['question1'],axis=1,inplace=True)
cs.drop(['Name'],axis=1,inplace=True)
cs

,Content,target
0,This guy that harassed me back in 2007 is hara...,0
1,Hi there I’m in my 30s and getting cyber bull...,0
2,Hi I was recently doxed by a few people who k...,0
3,I've had a group of ppl harassing me for over ...,0
4,Just wondering how this year went with everyon...,0
...,...,...
245,How do you get a book published?,1
246,Who is disrupting Bloomberg?,1
247,Why can flash run so fast?,1
248,What is the best way to invest $500 legally so...,1


In [17]:
cs['polarity'] = cs['Content'].map(lambda text: TextBlob(text).sentiment.polarity)
cs['sen_len'] = cs['Content'].astype(str).apply(len)
cs['word_count'] = cs['Content'].apply(lambda x: len(str(x).split()))

In [18]:
cs

,Content,target,polarity,sen_len,word_count
0,This guy that harassed me back in 2007 is hara...,0,-0.014132,1378,269
1,Hi there I’m in my 30s and getting cyber bull...,0,-0.004286,504,97
2,Hi I was recently doxed by a few people who k...,0,-0.095971,836,158
3,I've had a group of ppl harassing me for over ...,0,-0.056548,1197,220
4,Just wondering how this year went with everyon...,0,0.008333,270,45
...,...,...,...,...,...
245,How do you get a book published?,1,0.000000,32,7
246,Who is disrupting Bloomberg?,1,0.000000,28,4
247,Why can flash run so fast?,1,0.200000,26,6
248,What is the best way to invest $500 legally so...,1,0.400000,132,26


In [19]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)

In [20]:
cs['polarity'].iplot(
    kind='hist',
    bins=50,
    xTitle='polarity',
    linecolor='black',
    yTitle='count',
    title='Sentiment Polarity Distribution')

In [21]:
cs['sen_len'].iplot(
    kind='hist',
    bins=100,
    xTitle='review length',
    linecolor='black',
    yTitle='count',
    title='Review Text Length Distribution')

In [22]:
cs['word_count'].iplot(
    kind='hist',
    bins=100,
    xTitle='word count',
    linecolor='black',
    yTitle='count',
    title='Review Text Word Count Distribution')

In [17]:
id_child = ['child','children','kid','daughter','son','student','girl',
           'boy','18','17','16','15','14','13','12','11','10','Teens','school']

In [18]:
iter_1 = 0
id_list1 = []
for nm in cs['Content'].tolist():
    for id_ in id_child:
        if id_ in nm.lower().split():
            id_list1.append(iter_1)
        else:
            pass
    iter_1+=1
        

In [19]:
len(id_list1)

118

In [20]:
#extracting specific contents as per above :
f_list = list(id_list1)
len(f_list)

118

In [21]:
new_cs = cs.iloc[f_list].reset_index(drop=True) ; new_cs['target'] = [0 for x in range(new_cs.shape[0])]
new_cs.head(5),new_cs.tail(5)

(                                             Content  target
 0  Hi  I was recently doxed by a few people who k...       0
 1  Hi  I was recently doxed by a few people who k...       0
 2  Hi  This is my first post on this community so...       0
 3  Hi  This is my first post on this community so...       0
 4  I don't know anything about this site and my 1...       0,
                                                Content  target
 113  I have two daughters 8 and 14 and i am very wo...       0
 114  What are the laws to change your status from a...       0
 115  Is being a good kid and not being a rebel wort...       0
 116  In how many ways can we distribute 10 identica...       0
 117  In how many ways can we distribute 10 identica...       0)

Sentiment Analysis - Cybersmile [scrapped,filtered data]

In [22]:
corpus_cs_1 = cs['Content'].tolist()
target = cs['target'].tolist()

In [23]:
#stop words
english_stop_words = stopwords.words('english')
def remove_stop_words(corpus):
    removed_stop_words = []
    for review in corpus:
        removed_stop_words.append(
            ' '.join([word for word in review.split() 
                      if word not in english_stop_words])
        )
    return removed_stop_words

In [24]:
corpus_cs_1 = remove_stop_words(corpus_cs_1)

In [25]:
#lematize 
def get_lemmatized_text(corpus):
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    return [' '.join([lemmatizer.lemmatize(word) for word in review.split()]) for review in corpus]

corpus_cs_1 = get_lemmatized_text(corpus_cs_1)

Models - I

In [26]:
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(corpus_cs_1)
X = ngram_vectorizer.transform(corpus_cs_1)
X_test = ngram_vectorizer.transform(corpus_cs_1)

X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))
    
    
final_ngram = LogisticRegression(C=0.5)
final_ngram.fit(X, target)
print ("Logistic Regression Final Accuracy: %s" 
       % accuracy_score(target, final_ngram.predict(X_test)))

Accuracy for C=0.01: 0.904
Accuracy for C=0.05: 0.936
Accuracy for C=0.25: 0.944
Accuracy for C=0.5: 0.96
Accuracy for C=1: 0.96
Logistic Regression Final Accuracy: 0.9979959919839679


In [69]:
cm = metrics.confusion_matrix(target, lr.predict(X_test))
print(cm)

[[240   9]
 [  0 250]]


In [70]:
print(classification_report(target, lr.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      0.96      0.98       249
           1       0.97      1.00      0.98       250

    accuracy                           0.98       499
   macro avg       0.98      0.98      0.98       499
weighted avg       0.98      0.98      0.98       499



In [85]:
lr.coef_ , lr.intercept_

(array([[-1.01165046e-05, -1.01165046e-05,  0.00000000e+00, ...,
         -1.04197761e-01, -1.04197761e-01, -1.04197761e-01]]),
 array([2.32849639]))

In [71]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(corpus_cs_1)
X = tfidf_vectorizer.transform(corpus_cs_1)
X_test = tfidf_vectorizer.transform(corpus_cs_1)

X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr1 = LogisticRegression(C=c)
    lr1.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr1.predict(X_val))))

final_tfidf = LogisticRegression(C=1)
final_tfidf.fit(X, target)
print ("TF-IDF Final Accuracy: %s" 
       % accuracy_score(target, final_tfidf.predict(X_test)))

Accuracy for C=0.01: 0.64
Accuracy for C=0.05: 0.984
Accuracy for C=0.25: 0.976
Accuracy for C=0.5: 0.976
Accuracy for C=1: 0.976
TF-IDF Final Accuracy: 0.9959919839679359


In [72]:
cm = metrics.confusion_matrix(target, lr1.predict(X_test))
print(cm)

[[247   2]
 [  1 249]]


In [73]:
print(classification_report(target, lr1.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       249
           1       0.99      1.00      0.99       250

    accuracy                           0.99       499
   macro avg       0.99      0.99      0.99       499
weighted avg       0.99      0.99      0.99       499



In [86]:
lr1.coef_ , lr1.intercept_

(array([[-0.0221435 ,  0.        , -0.31650852, ..., -0.19314841,
         -0.0965742 , -0.19314841]]),
 array([0.51074244]))

In [74]:
ngram_vectorizer = CountVectorizer(binary=True, ngram_range=(1, 2))
ngram_vectorizer.fit(corpus_cs_1)
X = ngram_vectorizer.transform(corpus_cs_1)
X_test = ngram_vectorizer.transform(corpus_cs_1)

X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    svm = LinearSVC(C=c)
    svm.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, svm.predict(X_val))))

    
final_svm_ngram = LinearSVC(C=0.01)
final_svm_ngram.fit(X, target)
print ("SVM Final Accuracy: %s" 
       % accuracy_score(target, final_svm_ngram.predict(X_test)))

Accuracy for C=0.01: 0.968
Accuracy for C=0.05: 0.968
Accuracy for C=0.25: 0.968
Accuracy for C=0.5: 0.976
Accuracy for C=1: 0.976
SVM Final Accuracy: 0.9859719438877755


In [75]:
cm = metrics.confusion_matrix(target, svm.predict(X_test))
print(cm)

[[246   3]
 [  0 250]]


In [76]:
print(classification_report(target, svm.predict(X_test)))

              precision    recall  f1-score   support

           0       1.00      0.99      0.99       249
           1       0.99      1.00      0.99       250

    accuracy                           0.99       499
   macro avg       0.99      0.99      0.99       499
weighted avg       0.99      0.99      0.99       499



In [87]:
svm.coef_ , svm.intercept_

(array([[0., 0., 0., ..., 0., 0., 0.]]), array([0.54710676]))